In [1]:
from pyiceberg.catalog import load_catalog
import pyarrow.parquet as pq
import os
from pathlib import Path
import geopandas as gpd
from shapely import wkb
import numpy as np
import networkx as nx
from tqdm import tqdm

In [2]:
from pyiceberg.catalog import load_catalog

warehouse_path = "../data/warehouse"
catalog = load_catalog(
    "default",
    **{
        "type": "sql",
        "uri": f"sqlite:///{warehouse_path}/icefabric_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

In [5]:
def add_parquet_to_catalog(file_path, table_name):
    # Check if table already exists
    if catalog.table_exists(f"hydrofabric.{table_name}"):
        print(f"Table {table_name} already exists, loading it")
        return catalog.load_table(f"hydrofabric.{table_name}")

    # Read the parquet file
    arrow_table = pq.read_table(file_path)

    # Create the table in the catalog
    iceberg_table = catalog.create_table(
        f"hydrofabric.{table_name}",
        schema=arrow_table.schema,
    )

    # Append the data to the table
    iceberg_table.append(arrow_table)

    print(f"Added {file_path} as table {table_name}")
    return iceberg_table

In [6]:
# Get all parquet files from the directory
parquet_dir = "../data/parquet"
parquet_files = list(Path(parquet_dir).glob("*.parquet"))

# Dictionary to store all tables
tables = {}

# Add each parquet file to the catalog
for parquet_file in parquet_files:
    table_name = parquet_file.stem  # Get filename without extension
    tables[table_name] = add_parquet_to_catalog(str(parquet_file), table_name)

Table network already exists, loading it
Table nexus already exists, loading it
Table flowpath-attributes already exists, loading it
Table divides already exists, loading it
Table pois already exists, loading it
Table flowpath-attributes-ml already exists, loading it
Table divide-attributes already exists, loading it
Table flowpaths already exists, loading it
Table hydrolocations already exists, loading it
Table lakes already exists, loading it


In [4]:
catalog.load_table("hydrofabric.d")

NoSuchTableError: Table does not exist: hydrofabric.d

In [8]:
tables["network"]

network(
  1: id: optional string,
  2: toid: optional string,
  3: divide_id: optional string,
  4: ds_id: optional double,
  5: mainstem: optional double,
  6: hydroseq: optional double,
  7: hf_source: optional string,
  8: hf_id: optional double,
  9: lengthkm: optional double,
  10: areasqkm: optional double,
  11: tot_drainage_areasqkm: optional double,
  12: type: optional string,
  13: vpuid: optional string,
  14: hf_hydroseq: optional double,
  15: hf_lengthkm: optional double,
  16: hf_mainstem: optional double,
  17: topo: optional string,
  18: poi_id: optional double,
  19: hl_uri: optional string
),
partition by: [],
sort order: [],
snapshot: Operation.APPEND: id=3803992390522077902, schema_id=0

In [7]:
print("Tables in the catalog:")
for table_id in catalog.list_tables("hydrofabric"):
    print(f"- {table_id}")

Tables in the catalog:
- ('hydrofabric', 'divide-attributes')
- ('hydrofabric', 'divides')
- ('hydrofabric', 'flowpath-attributes')
- ('hydrofabric', 'flowpath-attributes-ml')
- ('hydrofabric', 'flowpaths')
- ('hydrofabric', 'hydrolocations')
- ('hydrofabric', 'lakes')
- ('hydrofabric', 'network')
- ('hydrofabric', 'nexus')
- ('hydrofabric', 'pois')


In [40]:
def create_geodataframe(df):
    if "geometry" in df.columns:
        df["geometry"] = df["geometry"].apply(lambda x: wkb.loads(x) if x is not None else None)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:5070")
    return df

In [8]:
def find_origin(network_table, identifier, id_type="hl_uri"):
    """Find an origin point in the hydrofabric network.

    This function handles the case where multiple records match the identifier.
    It follows the R implementation to select a single origin point based on
    the minimum hydroseq value.
    """
    # Filter network table by the identifier
    if id_type == "hl_uri":
        row_filter = f"{id_type} == '{identifier}'"
    # elif id_type == "comid":
    #     row_filter = f"hf_id == {identifier}"
    # elif id_type == "id":
    #     row_filter = f"id == '{identifier}'"
    # elif id_type == "poi_id":
    #     row_filter = f"poi_id == '{identifier}'"
    else:
        raise ValueError(f"Identifier type {id_type} not supported")

    # Get all matching records
    origin_candidates = network_table.scan(row_filter=row_filter).to_pandas()

    if len(origin_candidates) == 0:
        raise ValueError(f"No origin found for {id_type}='{identifier}'")

    # Select relevant columns for the origin
    origin_cols = ["id", "toid", "vpuid", "topo", "hydroseq"]
    available_cols = [col for col in origin_cols if col in origin_candidates.columns]

    # Select only the relevant columns and drop duplicates
    origin = origin_candidates[available_cols].drop_duplicates()

    # Find the record with minimum hydroseq
    if "hydroseq" in origin.columns:
        # For consistency with R, check if there are unique hydroseq values
        if len(origin["hydroseq"].unique()) > 1:
            # Sort by hydroseq and take the minimum
            origin = origin.sort_values("hydroseq").iloc[0:1]

    # If we still have multiple records, it's a problem
    if len(origin) > 1:
        raise ValueError(f"Multiple origins found: {origin['id'].tolist()}")

    return origin

In [ ]:
identifier = "gages-01563500"
id_type = "hl_uri"
origin = find_origin(network_table, identifier, id_type)
origin

,id,toid,vpuid,topo,hydroseq
1,wb-87646,nex-87404,02,fl-nex,26923.0


In [33]:
network_df = tables["network"].scan().to_pandas()
nexus_df = tables["nexus"].scan().to_pandas()
flowpaths_df = tables["flowpaths"].scan().to_pandas()

terminal_id = origin["id"].iloc[0]
terminal_nexus = origin["toid"].iloc[0]

In [30]:
wb_rows = network_df[network_df["toid"].str.contains("wb-", na=False)]
nex_rows = network_df[network_df["toid"].str.contains("nex-", na=False)]
wb_rows

,id,toid,divide_id,ds_id,mainstem,hydroseq,hf_source,hf_id,lengthkm,areasqkm,tot_drainage_areasqkm,type,vpuid,hf_hydroseq,hf_lengthkm,hf_mainstem,topo,poi_id,hl_uri
84513,nex-1000,wb-1000,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,nexus,01,NaN,NaN,NaN,fl-nex,NaN,None
84514,nex-10000,wb-10000,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,nexus,01,NaN,NaN,NaN,fl-nex,NaN,None
84515,nex-10004,wb-10004,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,nexus,01,NaN,NaN,NaN,fl-nex,NaN,None
84516,nex-10008,wb-10008,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,nexus,01,NaN,NaN,NaN,fl-nex,NaN,None
84517,nex-1001,wb-1001,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,nexus,01,NaN,NaN,NaN,fl-nex,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461032,tnx-1000014142,wb-0,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,terminal,18,NaN,NaN,NaN,fl-nex,NaN,None
3461033,tnx-1000014143,wb-0,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,terminal,18,NaN,NaN,NaN,fl-nex,NaN,None
3461034,tnx-1000014144,wb-0,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,terminal,18,NaN,NaN,NaN,fl-nex,NaN,None
3461035,tnx-1000014145,wb-0,None,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,terminal,18,NaN,NaN,NaN,fl-nex,NaN,None
